## Lập trình xử lý dữ liệu - Nhóm 7: Đánh đâu lỗ đó
### Notebook Lấy dữ liệu và Tiền xử lý dữ liệu

### Dữ liệu chứng khoán
---

#### Khai báo thư viện

In [1]:
from vnstock import Vnstock
import pandas as pd
import json

stock = Vnstock(source="VCI").stock()
df_list_stock = pd.DataFrame(stock.listing.symbols_by_industries())
# df_list_post = pd.read_csv('cleaned_posts.csv')

2025-04-11 02:47:44 - vnstock.common.vnstock - INFO - Mã chứng khoán không được chỉ định, chương trình mặc định sử dụng VN30F1M
2025-04-11 02:47:44 - vnstock.common.data.data_explorer - INFO - Không phải là mã chứng khoán, thông tin công ty và tài chính không khả dụng.


#### Hàm lấy dữ liệu cụ thể
Những hàm sau lấy các thông tin về công ty đại diện cho các mã chứng khoán, lấy từ các nguồn nhờ thư viện `vnstock3`:
- `get_info_industry`: Lấy thông tin về tên công ty, ngành của công ty
- `get_info_overview`: Lấy thông tin về số lượng cổ đông, số lượng nhân viên, ...
- `get_info_finance`: Lấy thông tin về tài chính của công ty, các chỉ số...

In [2]:
def get_info_industry(symbol: str):
    # Lấy hàng chứa thông tin của mã cổ phiếu cần tìm
    df_row = df_list_stock[df_list_stock['symbol'] == symbol]
    df_row = df_row[['symbol', 'organ_name', 'icb_code2', 'icb_name2']]
    df_row.columns = ['symbol', 'name', 'indus_code', 'indus_name']

    return df_row

get_info_industry('VCI')

,symbol,name,indus_code,indus_name
1424,VCI,Công ty Cổ phần Chứng khoán Vietcap,8700,Dịch vụ tài chính


In [3]:
def get_info_overview(symbol: str):
    # Lấy thông tin tổng quan của mã cổ phiếu cần tìm, từ TCBS
    df_info_ovw = pd.DataFrame(Vnstock(show_log=False).stock(symbol=symbol, source='TCBS').company.overview())
    df_info_ovw['symbol'] = symbol
    df_info_ovw = df_info_ovw[['symbol', 'exchange', 'no_shareholders', 'no_employees', 'stock_rating']]

    # Fill NaN thành -1
    df_info_ovw = df_info_ovw.fillna(-1).infer_objects(copy=False)

    # Đổi tên cột
    df_info_ovw.columns = ['symbol', 'exc', 'm_stholder', 'm_employee', 'r_tcbsrating']
    
    return df_info_ovw

get_info_overview('VCI')


,symbol,exc,m_stholder,m_employee,r_tcbsrating
0,VCI,HOSE,15669,389,2.4


In [4]:
def get_info_finance(symbol: str):
    # Thử lấy thông tin tài chính từ VCI
    df_row = pd.DataFrame()
    try:
        df_row = pd.DataFrame(Vnstock(show_log=False).stock(symbol=symbol, source='VCI').finance.ratio(period='year', lang='vi'))
    except:
        pass
    
    df_info_finance = pd.DataFrame()

    # Lấy các chỉ tiêu cơ bản trong năm gần nhất (2023)
    df_row = df_row.head(1)

    LIST_KEYS = [(('Meta', 'CP')                                      , 'symbol' ), 
                 (('Meta', 'Năm')                                     , 'year'   ),
                 (('Chỉ tiêu định giá', 'Vốn hóa (Tỷ đồng)')          , 'm_cap'  ),     # m_cap
                 (('Chỉ tiêu định giá', 'Số CP lưu hành (Triệu CP)')  , 'm_share'),     # m_share
                 (('Chỉ tiêu định giá', 'P/B')                        , 'r_pb'   ),     # r_pb      = giá thị trường / BVPS
                 (('Chỉ tiêu định giá', 'P/E')                        , 'r_pe'   ),     # r_pe      = giá thị trường / lợi nhuận sau thuế
                 (('Chỉ tiêu định giá', 'EPS (VND)')                  , 'r_eps'  ),     # r_eps     = lợi nhuận sau thuế / số CP lưu hành
                 (('Chỉ tiêu định giá', 'BVPS (VND)')                 , 'r_bvps' ),     # r_bvps    = vốn chủ sở hữu / số CP lưu hành
                 (('Chỉ tiêu khả năng sinh lợi', 'ROE (%)')           , 'p_roe'  ),     # p_roe     = lợi nhuận sau thuế / vốn chủ sở hữu
                 (('Chỉ tiêu khả năng sinh lợi', 'Tỷ suất cổ tức (%)'), 'p_div'  ),     # p_div     = Tỷ suất cổ tức = cổ tức / giá thị trường
                ]

    # Kiểm tra xem các chỉ tiêu có trong df_info_finance không
    for key, col_name in LIST_KEYS:
        if key not in df_row.columns:
            df_row[key] = 0

        # Đổi tên cột thanh col_name
        df_info_finance[col_name] = df_row[key]

    return df_info_finance

get_info_finance('VCI')

,symbol,year,m_cap,m_share,r_pb,r_pe,r_eps,r_bvps,p_roe,p_div
0,VCI,2024,23015088334000,718099480,1.778009,25.272085,1268.197706,18025.783068,0.089655,0.020281


In [5]:
# Lấy toàn bộ mã cổ phiếu trong list_stock
list_symbol = df_list_stock['symbol'].tolist()
df_info = pd.DataFrame()

for i, symbol in enumerate(list_symbol):
    # Lấy thông tin của mã cổ phiếu ở 3 hàm
    df_info_industry = get_info_industry(symbol)
    df_info_ovw = get_info_overview(symbol)
    df_info_finance = get_info_finance(symbol)

    # Gộp
    df_info_temp = pd.merge(df_info_industry, df_info_ovw, on='symbol', how='inner')
    df_info_temp = pd.merge(df_info_temp, df_info_finance, on='symbol', how='inner')

    df_info = pd.concat([df_info, df_info_temp], ignore_index=True)

    # Lưu vào file CSV
    df_info_temp.to_csv('list_stock.csv', index=False, encoding='utf-8', header=(False if i > 0 else True), mode='a')
    print(f'Processed {symbol} ({i+1}/{len(list_symbol)})!', end='\r')

print(f'\nProcessed {len(df_info)} stocks, end of data')

df_info = df_info.reset_index(drop=True)
df_info.tail(20)

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
2025-04-11 02:47:48 - vnstock.explorer.vci.financial - ERROR - Error processing financial report data: "None of [Index(['CP', 'Năm', 'Kỳ'], dtype='object')] are in the [columns]"
2025-04-11 02:47:48 - vnstock.explorer.vci.financial - ERROR - Error retrieving financial ratios: "None of [Index(['CP', 'Năm', 'Kỳ'], dtype='object')] are in the [columns]"


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


KeyboardInterrupt: 

In [2]:
# Danh sách ngành
df_list_indus = df_list_stock[['icb_code2', 'icb_name2']].drop_duplicates().reset_index(drop=True)
df_list_indus

,icb_code2,icb_name2
0,8300,Ngân hàng
1,3500,Thực phẩm và đồ uống
2,2300,Xây dựng và Vật liệu
3,1300,Hóa chất
4,7500,"Điện, nước & xăng dầu khí đốt"
5,9500,Công nghệ Thông tin
6,5500,Truyền thông
7,2700,Hàng & Dịch vụ Công nghiệp
8,4500,Y tế
9,5700,Du lịch và Giải trí


#### Hàm lấy giá lịch sử

Hàm này sẽ lấy giá lịch sử của các mã được đề cập đến trong những bài viết trong file `cleaned_posts.csv`.
- Khung thời gian lấy sẽ là 30 phút, trong trường hợp không có đủ dữ liệu thì dùng khung 1 ngày.
- Bắt đầu lấy dữ liệu từ ngày `01/9/2024` cho đến `08/11/2024`.

In [11]:
def process_symbol(cmponent, symb, interval='1D'):
    try:
        df = pd.DataFrame(cmponent.quote.history(symbol=symb, start='2024-12-01', end='2025-04-11', interval = interval))
        if len(df) == 0:
            raise Exception('No data')
        
        # Xóa các dòng  không cần thiết
        df = df.dropna()

        # Lưu vào file CSV
        df.to_csv('price/' + symb + '.csv', index=False)
        return 1
    except Exception as e:
        print(f'\nError processing {symb}: {e}')
        return 0

In [6]:
# Lấy danh sách mã cổ phiếu được đề cập
# Công nghệ		    FPT
# Chứng khoán		    SSI
# Ngân hàng		    VCB
# Bất động sản		VHM
# Vật liệu cơ bản		HPG
# Dịch vụ Hạ tầng		GAS
# Tiêu dùng cơ bản	MSN
# Bán lẻ			    MWG
# Chế biến		    GVR
# Công nghiệp		    VCG


list_taggedSymbols = ["FPT", "SSI", "VCB", "VHM", "HPG", "GAS", "MSN", "MWG", "GVR", "VCG"]
# set_tagg

# for post in df_list_post['taggedSymbols']:
#     symb_array = json.loads(post)
#     for symb in symb_array:
#         set_taggedSymbols.add(symb['symb'])

# list_taggedSymbols = list(set_taggedSymbols)
list_taggedSymbols.sort()

print(list_taggedSymbols)

['FPT', 'GAS', 'GVR', 'HPG', 'MSN', 'MWG', 'SSI', 'VCB', 'VCG', 'VHM']


In [12]:
# Kiểm tra xem thư mục price có tồn tại không, nếu không thì tạo mới
import os
if not os.path.exists('price'):
    os.makedirs('price')

count_processed = 0

for i, symb in enumerate(list_taggedSymbols):

    # # Phân loại mã cổ phiếu
    # if symb[0] == '$':  # Tiền điện tử (crypto)
    #     crypto = Vnstock().crypto(source='MSN')
    #     count_processed += process_symbol(crypto, symb[1:])
    # elif symb[0] == '^':  # Chỉ số (index)
    #     index = Vnstock().world_index(source='MSN')
    #     count_processed += process_symbol(index, symb[1:])
    # elif '-' in symb:     # Tỷ giá (forex)
    #     forex = Vnstock().fx(source='MSN')
    #     count_processed += process_symbol(forex, symb.replace('-', ''))
    # elif len(symb) > 3 and symb[0] == 'C': # Chứng quyền (warrant), bỏ qua
    #     continue
    # elif len(symb) > 3 and (symb[-2::] == '24' or symb[-2::] == '25'): # Hợp đồng tương lai (future), bỏ qua
    #     continue
    # elif '=' in symb or '.' in symb:     # Một số mã cổ phiếu khác, bỏ qua
    #     continue
    # else:
    count_processed += process_symbol(stock, symb) # Mã cổ phiếu thông thường, lấy dữ liệu 30 phút
    print(f'Processed {symb} ({i+1}/{len(list_taggedSymbols)}), saved to price/1d/{symb}.csv', end='\r')

print(f'\nProcessed {count_processed} stocks, end of data')


Processed VHM (10/10), saved to price/1d/VHM.csv
Processed 10 stocks, end of data
